# 0. Import libraries

In [1]:
import os
import hashlib
from datetime import datetime
from collections import defaultdict

# 1. Check image and label pairs

In [2]:
def find_files_with_same_basename(directory):
    # 用于存储文件名和对应的扩展名
    file_dict = defaultdict(set)

    # 遍历指定目录下的所有文件
    for root, _, files in os.walk(directory):
        for file in files:
            basename, ext = os.path.splitext(file)
            file_dict[basename].add(ext.lstrip("."))  # 去掉扩展名前面的点

    # 构建结果列表
    result = [{"basename": basename, "extensions": list(extensions)} for basename, extensions in file_dict.items() if len(extensions) > 1]
    
    return result

# 2. Rename files

In [3]:
def rename_files(file_groups, directory, naming_method="timestamp", start_index=1):
    # 用于存储重命名后的文件信息
    renamed_files = []

    # 记录时间戳用于生成名字
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    for index, group in enumerate(file_groups, start=start_index):
        basename = group["basename"]
        extensions = group["extensions"]

        # 根据不同的命名方式生成新文件名
        if naming_method == "timestamp":
            # 使用时间戳+序号生成名字
            new_basename = f"{timestamp}_{index}"
        elif naming_method == "hash":
            # 使用hash生成名字
            new_basename = hashlib.md5(basename.encode()).hexdigest()
        else:
            raise ValueError("不支持的命名方式")

        # 遍历该组的所有扩展名并重命名
        for ext in extensions:
            old_file = os.path.join(directory, f"{basename}.{ext}")
            new_file = os.path.join(directory, f"{new_basename}.{ext}")

            # 进行重命名
            os.rename(old_file, new_file)

            # 将旧文件路径与新文件路径映射存储
            renamed_files.append({
                "old_file": old_file,
                "new_file": new_file
            })

    return renamed_files

# 3. Main

In [4]:
# 请求用户输入路径
directory = input("请输入文件夹的路径: ")

# 检查路径是否存在
if os.path.exists(directory):
    file_groups = find_files_with_same_basename(directory)
    rename_files(file_groups, directory, naming_method="hash")
else:
    print("指定的文件夹不存在")